# チュートリアル1: DQNの使い方を理解しよう

## このノードブックの目的
- pytorch で構築したサンプル用DQNモデルを使って、実際に推論を試し、サンプル用モデルの入出力形式を理解する。

# 序章. ブロックを可視化する関数を準備しよう!
- 事前準備としてnumpy の配列をテトリスの盤面のように可視化する関数を準備します。
- この関数は実際のコードでは使用しておらず、本チュートリアルにおいて理解を深めるためのものです。

## 0.1. matplotlibのインストール 
可視化用のライブラリ[matplotlib](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html)をインストールします。

In [ ]:
!pip install matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

## 0.2. サンプル用テストデータの準備
- AIモデルに入力するサンプル用テストデータをロードします。
- チュートリアルとして10パターンのテストデータを格納した .npz ファイルを用意しています。

In [ ]:
import numpy as np
tutorial_data = np.load("./test_data10.npz") #1~10までのサンプル用状態データをロード

In [ ]:
colors = [[255,255,255],
          [255,0,0],
          [0,255,0],
          [255,255,0], 
          [218, 179, 0],
          [247, 171, 166],
          [0,0,255],
          [255,255,0]]

#ブロック可視化関数 (numpy 配列からテトリスの盤面を可視化)
def visualize_block(data, shape, size=0.5):
    block_array = []
    fig, ax = plt.subplots()
    fig.set_figwidth(shape[0] * size)
    fig.set_figheight(shape[1] * size)
    for i in range(shape[0]):
        row = []
        for j in range(shape[1]):
            c = colors[int(data[i][j])]
            row.append(c)
        block_array.append(row)
    block_array = np.array(block_array)
    im = plt.imshow(block_array)

    #グリッド線を引く
    ax.set_xticks(np.arange(-0.5, 10, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 23, 1.0), minor=True)
    ax.set_xticks(np.arange(-0.5, 10,10))
    ax.set_yticks(np.arange(-0.5, 23, 3))
    ax.grid(which='minor', color='black', linestyle='-', linewidth=0.5)
    ax.grid(which='major', color='black', linestyle='-', linewidth=0.5)
    ax.imshow(block_array)
    
shape = tutorial_data["1"].shape 
visualize_block(tutorial_data["1"],shape) #サンプル用状態データ１を可視化


In [ ]:
import sys
sys.path.append("../")
sys.path.append("../../../")
from deepqnet import DeepQNetwork
import torch
import os

In [ ]:
# 2値関数
def get_reshape_backboard(board):
    reshape_board = np.where(board > 0, 1, 0)
    return reshape_board

# 1章 AIモデル(DQN)に状態の価値を推論させよう！
- ここからは上記で準備したサンプル用テストデータを用いて、AIモデル(DQN)に推論をさせてみましょう。
- DQN では状態を　22x10　のテンソルとして表現します。

## 1.1 DQNモデルをロード
- DQNモデルをロードしましょう。　
- pytorch では　torch.load() で学習済みの重みファイルをロードし、モデルを構築できます。
- 今回はサンプル用の重みファイル　"tetris/weight/DQN/sample_weight.pt" を使ってみてください。

In [ ]:
model = torch.load("../../../../weight/DQN/sample_weight.pt") #モデルをロード

## 1.2テストデータのロード
- 実際にモデルを使って状態（テトリスの盤面）の価値を推定するためのテストデータをロードしましょう。
- 0.2. で準備したサンプルデータを使って、モデルの入出力がどのような形式かを確認してみてください。

In [ ]:
# ###index を1~10の番号に変えてみてください###

data_index =  1

# #######################################

tutorial_data = np.load("./test_data10.npz") #1~10までのサンプル用状態データをロード
load_data = tutorial_data[repr(data_index)] 
shape = load_data.shape
visualize_block(load_data ,shape) #データを可視化

- サンプル用DQNモデルではshape=(batch_size,22,10)のテンソルを入力とします。
- また、状態のテンソルの各要素は0または1になります.（0: ブロックなし、1:ブロックあり)
- 下記の手順で入力の形式を合わせます。
    1. get_reshape_backboard()を用いて状態のデータを1,0の２値データ化
    2. 複数の状態データをまとめる　（今回は１つ）
    3. numpy を tensor形式に変換
    
- 今回は１つの状態データをテンソル化するのでリストで３Dにした後、torch.from_numpy()　を使っていますが、
複数のバッチで入力する場合は torch.stack()　を使うと便利です。

In [ ]:
state = get_reshape_backboard(load_data) # 1,0 の2値データに変更
print(state)
states =np.array([state]) # 3Dに変更　(batch_size,Height,Width) 今回はbatch_size=1とする
next_states = torch.from_numpy(states.astype(np.float32)).clone() #numpy -> tensor 型に変更

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(next_states) #状態の価値を推論　(modelのforward()が実行される)
print(predictions) #状態の価値（大きい方が価値が高い）

上記で出力された結果が、入力状態に対してモデルが推論した価値になります。

例えばサンプルデータの1番(data_index=1)では価値が　-0.1795 に対して サンプルデータの8番(data_index=8)では価値が　-2.6101 となります。

これはサンプルデータの1番が表す状態の方が価値が高い＝将来的に報酬を得やすいことを表しています。